# Scraper eBay Asynchrone avec Jupyter Notebook

Ce notebook implémente un scraper eBay asynchrone en Python en utilisant les bibliothèques suivantes :

- `aiohttp` et `asyncio` pour les requêtes HTTP asynchrones
- `BeautifulSoup` pour le parsing HTML
- `fake_useragent` pour la rotation des User-Agent
- `csv` pour l'export des résultats
- `datetime` et `os` pour la gestion des dates et fichiers

Le script effectue les opérations suivantes :
- Recherche de produits pour différentes catégories (Laptops, Monitors, Smart Watches, Graphics Cards)
- Récupération des liens des produits sur plusieurs pages de recherche
- Scraping des détails de chaque produit
- Sauvegarde des résultats dans des fichiers CSV (un par catégorie)

> **Note :** Dans un environnement Jupyter, comme un event loop est déjà en cours, nous utilisons la bibliothèque `nest_asyncio` pour autoriser l'exécution de boucles asynchrones imbriquées.


In [ ]:
# %% [code]
# Importation des librairies nécessaires
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import random
from fake_useragent import UserAgent
from datetime import datetime
import os


#
#  Définition des fonctions utilitaires
La fonction `get_headers()` permet de générer des en-têtes HTTP avec un User-Agent aléatoire pour rendre les requêtes moins facilement détectables par eBay.


In [ ]:
# %% [code]
# Initialisation de fake_useragent
ua = UserAgent()

def get_headers():
    return {
        'User-Agent': ua.random,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Referer': 'https://www.ebay.com/',
        'DNT': '1'
    }



#  Fonctions de Scraping

 Nous définissons ici plusieurs fonctions asynchrones pour :

 - **Scraper les détails d'un produit :** `scrape_product_details()`
 Cette fonction récupère le titre, le prix et d'autres caractéristiques en fonction de la catégorie.

 - **Scraper une page de résultats de recherche :** `scrape_search_page()`
 Elle récupère les URL des produits listés sur la page de recherche.

 - **scraping plusieurs pages pour chaque catégorie :** `scrape_ebay_search()`
   Pour chaque catégorie, cette fonction exécute le scraping sur plusieurs pages puis collecte les détails des produits.


In [ ]:

async def scrape_product_details(session, product_url, category):
    try:
        # Attendre un délai aléatoire pour simuler un comportement humain
        await asyncio.sleep(random.uniform(2, 5))
        headers = get_headers()

        async with session.get(product_url, headers=headers) as response:
            response.raise_for_status()
            soup = BeautifulSoup(await response.text(), 'html.parser')

            # Extraction du titre du produit
            title = soup.find('h1', class_='x-item-title__mainTitle')
            title = title.text.strip() if title else 'N/A'

            # Extraction du prix
            price = soup.find('div', class_='x-price-primary')
            price = price.text.strip() if price else 'N/A'

            # Extraction des spécifications
            specs = {}
            for spec in soup.find_all('div', class_='ux-labels-values__labels'):
                key = spec.text.strip()
                # Recherche de la valeur associée à la clé
                value_tag = spec.find_next('div', class_='ux-labels-values__values')
                value = value_tag.text.strip() if value_tag else 'N/A'
                specs[key] = value

            # Dictionnaire de base avec les informations communes
            product_details = {
                'Title': title,
                'Price': price,
                'Collection Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            # Enrichissement des données en fonction de la catégorie
            if category == "Laptops":
                product_details.update({
                    'RAM': specs.get('RAM Size', 'N/A'),
                    'CPU': specs.get('Processor', 'N/A'),
                    'Model': specs.get('Model', 'N/A'),
                    'Brand': specs.get('Brand', 'N/A'),
                    'GPU': specs.get('GPU', 'N/A'),
                    'Screen Size': specs.get('Screen Size', 'N/A'),
                    'Storage': specs.get('SSD Capacity', 'N/A'),
                })
            elif category == "Monitors":
                product_details.update({
                    'Screen Size': specs.get('Screen Size', 'N/A'),
                    'Maximum Resolution': specs.get('Resolution', 'N/A'),
                    'Aspect Ratio': specs.get('Aspect Ratio', 'N/A'),
                    'Refresh Rate': specs.get('Refresh Rate', 'N/A'),
                    'Response Time': specs.get('Response Time', 'N/A'),
                    'Brand': specs.get('Brand', 'N/A'),
                    'Model': specs.get('Model', 'N/A'),
                })
            elif category == "Smart Watches":
                product_details.update({
                    'Case Size': specs.get('Case Size', 'N/A'),
                    'Battery Capacity': specs.get('Battery Capacity', 'N/A'),
                    'Brand': specs.get('Brand', 'N/A'),
                    'Model': specs.get('Model', 'N/A'),
                    'Operating System': specs.get('Operating System', 'N/A'),
                    'Storage Capacity': specs.get('Storage Capacity', 'N/A')
                })
            elif category == "Graphics Cards":
                product_details.update({
                    'Brand': specs.get('Brand', 'N/A'),
                    'Memory Size': specs.get('Memory Size', 'N/A'),
                    'Memory Type': specs.get('Memory Type', 'N/A'),
                    'Chipset/GPU Model': specs.get('Chipset/GPU Model', 'N/A'),
                    'Connectors': specs.get('Connectors', 'N/A')
                })

            print(f"Successfully scraped {category}: {title[:50]}...")
            return product_details

    except Exception as e:
        print(f"Error scraping {product_url}: {str(e)}")
        return None

async def scrape_search_page(session, query, page, semaphore, category):
    async with semaphore:
        try:
            base_url = "https://www.ebay.com/sch/i.html"
            params = {'_nkw': query, '_sacat': 0, '_from': 'R40', '_pgn': page}
            headers = get_headers()

            async with session.get(base_url, params=params, headers=headers) as response:
                response.raise_for_status()
                soup = BeautifulSoup(await response.text(), 'html.parser')

                # Récupération de tous les éléments produits de la page
                items = soup.find_all('div', class_='s-item__wrapper')
                product_urls = [item.find('a', class_='s-item__link')['href']
                                for item in items if item.find('a', class_='s-item__link')]

                print(f"Scraped page {page} for {category} ({len(product_urls)} products)")
                return product_urls

        except Exception as e:
            print(f"Error scraping page {page} for {category}: {str(e)}")
            return []

async def scrape_ebay_search(categories, max_pages=1):
    all_products = {}
    semaphore = asyncio.Semaphore(2)

    async with aiohttp.ClientSession() as session:
        for category, query in categories.items():
            print(f"\n{'=' * 30}\nStarting {category} scraping\n{'=' * 30}")
            tasks = [scrape_search_page(session, query, page, semaphore, category)
                     for page in range(1, max_pages + 1)]
            search_results = await asyncio.gather(*tasks)
            product_urls = [url for sublist in search_results for url in sublist]

            product_tasks = [scrape_product_details(session, url, category) for url in product_urls]
            products = await asyncio.gather(*product_tasks)

            all_products[category] = [p for p in products if p]
            print(f"\n{'=' * 30}\nCompleted {category} ({len(all_products[category])} items)\n{'=' * 30}")

    return all_products



## Sauvegarde des données dans des fichiers CSV

 Une fois les données extraites, nous souhaitons les sauvegarder dans des fichiers CSV afin de faciliter leur exploitation ultérieure.

Les fonctions suivantes permettent :
- de déterminer le numéro de scrape suivant (`get_next_scrape_number`),
 - de sauvegarder les données dans le dossier correspondant à la catégorie (`save_to_csv`).


In [ ]:
# %% [code]
def get_next_scrape_number(save_directory, category):
    """Détermine le prochain numéro de scrape pour un dossier donné."""
    scrape_number = 1
    for filename in os.listdir(save_directory):
        if filename.startswith(f"{category}_") and filename.endswith(".csv"):
            try:
                # Extraction du numéro de scrape depuis le nom de fichier
                current_number = int(filename.split('_scrape')[-1].split('.')[0])
                if current_number >= scrape_number:
                    scrape_number = current_number + 1
            except ValueError:
                continue
    return scrape_number

def save_to_csv(data, category, save_directory, fieldnames):
    # Formatage du nom de dossier et du fichier
    category_folder = category.lower().replace(' ', '_')
    category_filename = category_folder
    today_date = datetime.now().strftime('%Y_%m_%d')

    # Création du dossier de sauvegarde s'il n'existe pas
    category_directory = os.path.join(save_directory, category_folder)
    os.makedirs(category_directory, exist_ok=True)

    # Détermination du prochain numéro de scrape
    scrape_number = get_next_scrape_number(category_directory, category_filename)

    # Génération du nom de fichier
    filename = os.path.join(category_directory, f"{category_filename}_{today_date}_scrape{scrape_number}.csv")

    # Sauvegarde des données en CSV
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

    print(f"Saved {len(data)} {category} items to {filename}")


 %% [markdown]
 ## Fonction principale et exécution du scraping

 La fonction `main()` orchestre l’ensemble du processus :
 1. Pour chaque catégorie (Laptops, Monitors, Smart Watches, Graphics Cards), on scrape les pages de recherche.
 2. On récupère les URLs des produits, puis on extrait les détails de chacun.
 3. On sauvegarde ensuite les données dans un fichier CSV dédié à chaque catégorie.

 **Note pour les notebooks Jupyter :**
 Dans certains cas, un événement loop est déjà en cours. Si vous rencontrez une erreur avec `asyncio.run(main())`, vous pouvez installer et utiliser la librairie `nest_asyncio` de la manière suivante :
 ```python
 !pip install nest_asyncio
 import nest_asyncio
 nest_asyncio.apply()
 await main()
 ```


In [ ]:
# %% [code]
async def main():
    categories = {
        "Laptops": "laptop",
        "Monitors": "monitor",
        "Smart Watches": "smart watch",
        "Graphics Cards": "graphics card"
    }

    max_pages = 18  # Nombre de pages à scraper par catégorie
    save_directory = "data/raw/ebay"

    print("\nStarting eBay scraping...")
    all_products = await scrape_ebay_search(categories, max_pages)

    # Champs (colonnes) attendus pour chaque catégorie
    category_fields = {
        "Laptops": ['Title', 'Price', 'RAM', 'CPU', 'Model', 'Brand', 'GPU', 'Screen Size', 'Storage', 'Collection Date'],
        "Monitors": ['Title', 'Price', 'Screen Size', 'Maximum Resolution', 'Aspect Ratio', 'Refresh Rate', 'Response Time', 'Brand', 'Model', 'Collection Date'],
        "Smart Watches": ['Title', 'Price', 'Case Size', 'Battery Capacity', 'Brand', 'Model', 'Operating System', 'Storage Capacity', 'Collection Date'],
        "Graphics Cards": ['Title', 'Price', 'Brand', 'Memory Size', 'Memory Type', 'Chipset/GPU Model', 'Connectors', 'Collection Date']
    }

    # Sauvegarde des données pour chaque catégorie
    for category, products in all_products.items():
        if products:
            save_to_csv(products, category, save_directory, category_fields[category])

